<a href="https://colab.research.google.com/github/Armadillobambi/NLP/blob/main/QA_SQuAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-transformers
import torch
import pandas as pd
from transformers import pipeline
from pytorch_transformers import RobertaTokenizer
from pytorch_transformers import RobertaModel

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_json('/content/drive/MyDrive/NLP/train-v2.0(1).json')
df.head()

In [ ]:
# Create configuration object holds information (hyperparameters) concerning the model

# from: https://medium.com/analytics-vidhya/6-steps-to-build-roberta-a-robustly-optimised-bert-pretraining-approach-e508ebe78b96
# change paths and tune values !

class Config(dict):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    for k, v in kwargs.items():
      setattr(self, k, v)

  def set(self, key, val):
    self[key] = val
    setattr(self, key, val)

config = Config(
    '''
    seed: how to split the data in training and validation set
    max_lr: maximum learning rate
    epochs: number of epoch
    batch_size: set to 4 because of gpu memory limit
    max_seq_len: max length of tokens in a sentence
    hidden_dropout_prob: the percentage of dropout
    hidden_size: 1024 for roberta-large and 768 for roberta-base
    valid_pct: percentage of validation dataset
    start_tok: start of sentence
    end_tok: end of sentence
    '''
    seed = 18,
    roberta_model_name = 'roBERTa',
    max_lr = 1e-5,
    max_epochs = 1,
    max_steps = -1                        # takes precedence over max_epochs
    precision = 16,
    batch_size = 4,
    max_seq_len = 320,
    hidden_dropout_prob = 0.05,
    hidden_size = 1024,
    valid_pct = 0.20,
    start_tok = "<s>",
    end_tok = "</s>",
    model_path = 'Model_Roberta.pkl',
    pred_path = 'Prediction_Roberta.csv',
    train_file_path = 'train.csv',
    test_file_paht = 'test.csv',
    text_column_nmae = 'question',
    target_column_name = 'answer'
)




Do pre-processing and set up tokenizer

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")

Import and configure model

In [ ]:
from transformers import RobertaModel
model = RobertaModel.from_pretrained('roberta-base')